<a href="https://colab.research.google.com/github/rahuljungbahadur/hands_on_ml_book/blob/main/chp12_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercises From chapter 12

## Do you get the same result with tf.range(10) and tf.constant(np.arange(10))?

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
tf.range(10)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [ ]:
tf.constant(np.arange(10))

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>

The later one has a default data type of `int64` (higher precision). Whereas the tensorflow one has a dtype of `int32` (lower precision)

## Sample python function converted to tensorlfow

In [ ]:
@tf.function
def sum_squares(n):
  s=0
  for i in range(n):
    s+=i**2
  
  return s


In [ ]:
sum_squares  ## It is a python funcition

In [ ]:
tf.autograph.to_code(sum_squares.python_function)

"def tf__sum_squares(n):\n    with ag__.FunctionScope('sum_squares', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:\n        do_return = False\n        retval_ = ag__.UndefinedReturnValue()\n        s = 0\n\n        def get_state():\n            return (s,)\n\n        def set_state(vars_):\n            nonlocal s\n            (s,) = vars_\n\n        def loop_body(itr):\n            nonlocal s\n            i = itr\n            s = ag__.ld(s)\n            s += (i ** 2)\n        i = ag__.Undefined('i')\n        ag__.for_stmt(ag__.converted_call(ag__.ld(range), (ag__.ld(n),), None, fscope), None, loop_body, get_state, set_state, ('s',), {'iterate_names': 'i'})\n        try:\n            do_return = True\n            retval_ = ag__.ld(s)\n        except:\n            do_return = False\n            raise\n        return fscope.ret(retval_, do_return)\n"

In [ ]:
def tf__sum_squares(n):
    with ag__.FunctionScope('sum_squares', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        s = 0

        def get_state():
            return (s,)

        def set_state(vars_):
            nonlocal s
            (s,) = vars_

        def loop_body(itr):
            nonlocal s
            i = itr
            s = ag__.ld(s)
            s += (i ** 2)
        i = ag__.Undefined('i')
        ag__.for_stmt(ag__.converted_call(ag__.ld(range), (ag__.ld(n),), None, fscope), None, loop_body, get_state, set_state, ('s',), {'iterate_names': 'i'})
        try:
            do_return = True
            retval_ = ag__.ld(s)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)


This way you can see the tensorflow generated code

## Autodiff example

In [19]:
def f(w1, w2):
  return 3* w1**2 + 2*w1*w2

In [20]:
f(2,3)

24

In [18]:
f(tf.Variable(5), tf.Variable(3)) ## Same result

<tf.Tensor: shape=(), dtype=int32, numpy=105>

In [21]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)

with tf.GradientTape() as tape:
  z=f(w1, w2)

gradients=tape.gradient(z, [w1, w2])

In [22]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [30]:
def loger(x):
  return tf.math.sin(x)

In [31]:
w3 = tf.Variable(1.)

with tf.GradientTape() as tape:
  q = loger(w3)

grads = tape.gradient(q, [w3])

In [32]:
grads

[<tf.Tensor: shape=(), dtype=float32, numpy=0.5403023>]

## Implement a custom layer that performs Layer Normalization (we will use this type of layer in Chapter 15):

In [81]:
## Custom layer
class LayerNorm(tf.keras.layers.Layer):

  ## Initialization
  def __init__(self, **kwargs) -> None:
      super().__init__(**kwargs)
      # self.units=units

  def build(self, input_shape):

    alpha_init=tf.ones_initializer()

    self.alpha = tf.Variable(initial_value=alpha_init(shape=(input_shape[-1:])), 
                             trainable=True,
                             name='alpha',                             
                             dtype=tf.float32)
    
    beta_init = tf.zeros_initializer()

    self.beta = tf.Variable(initial_value=beta_init(shape=(input_shape[-1:])),
                            trainable=True,
                            dtype=tf.float32,
                            name='beta')

    super().build(input_shape)

  ## Call method
  def call(self, X):
      mean, var = tf.nn.moments(X, axes=-1, keepdims=True)
      sd = tf.sqrt(var)

      numerator = (X-mean)
      denominator = sd + tf.constant(0.001)

      return tf.math.multiply(self.alpha, (numerator/denominator)) + self.beta

  # def compute_output_shape(self, input_shape):
  #     return input_shape

  # def get_config(self):
  #     base_config = super().get_config()
  #     return {**base_config}

    
  
    

    



In [64]:
from tensorflow.keras.layers import LayerNormalization

In [85]:
custom_layer = LayerNorm()
actual_layer = LayerNormalization()

In [67]:
sample_matrix = tf.Variable(initial_value=[[1,2], [3,4]], dtype=tf.float32)

In [68]:
sample_matrix

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

In [70]:
sample_val = tf.constant(2, dtype=tf.float32)

In [71]:
actual_layer(sample_matrix)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-0.99800605,  0.99800605],
       [-0.99800605,  0.99800605]], dtype=float32)>

In [86]:
custom_layer(sample_matrix)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-0.998004,  0.998004],
       [-0.998004,  0.998004]], dtype=float32)>

In [87]:
tf.reduce_mean(tf.losses.mean_absolute_error(custom_layer(sample_matrix), actual_layer(sample_matrix)))

<tf.Tensor: shape=(), dtype=float32, numpy=2.026558e-06>